In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# In-built dataset
import tensorflow.compat.v1 as tf
#tf.enable_eager_execution()
tf.executing_eagerly()
tf.disable_v2_behavior()
import tensorflow_datasets as tfds

# TensorBoard : load notebook extension and clear previous logs
import datetime
%load_ext tensorboard
!rm -rf ./logs/

batch_size = 128
num_classes = 54
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28


# batch_size=-1 to get the full dataset in NumPy arrays from the returned tf.Tensor object
emnist_train = tfds.load(name="emnist", split=tfds.Split.TRAIN, batch_size=-1) 
emnist_test = tfds.load(name="emnist", split=tfds.Split.TEST, batch_size=-1)

# tfds.as_numpy return a generator that yields NumPy array records out of a tf.data.Dataset
emnist_train = tfds.as_numpy(emnist_train) 
emnist_test = tfds.as_numpy(emnist_test)

x_train, y_train = emnist_train["image"], emnist_train["label"] # seperate x and y
x_test, y_test = emnist_test["image"], emnist_test["label"]


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train shape: (697932, 28, 28, 1)
697932 train samples
116323 test samples


In [4]:
# convert class vectors to binary class matrices
num_classes = 62
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [5]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_split=0.2,
          #validation_data=(x_test, y_test),
          callbacks=[tensorboard_callback])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train on 558345 samples, validate on 139587 samples
Epoch 1/12
558345/558345 [==============================] - 925s 2ms/step - loss: 0.7513 - accuracy: 0.7707 - val_loss: 0.4434 - val_accuracy: 0.8447
Epoch 2/12
558345/558345 [==============================] - 900s 2ms/step - loss: 0.5356 - accuracy: 0.8237 - val_loss: 0.4264 - val_accuracy: 0.8531
Epoch 3/12
558345/558345 [==============================] - 941s 2ms/step - loss: 0.5041 - accuracy: 0.8325 - val_loss: 0.4182 - val_accuracy: 0.8558
Epoch 4/12
558345/558345 [==============================] - 868s 2ms/step - loss: 0.4876 - accuracy: 0.8369 - val_loss: 0.4083 - val_accuracy: 0.8589
Epoch 5/12
558345/558345 [==============================] - 854s 2ms/step - loss: 0.4773 - accuracy: 0.8397 - val_loss: 0.3933 - val_accuracy: 0.8584
Epoch 6/12
558345/558345 [==============================] - 856s 2ms/step - loss: 0.4707 - accuracy: 0.8416 - val_loss: 0.4046 - val_accuracy: 0.8616
Epoch 7/12
558345/558345 [======================

## TensorBoard

In [7]:
tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 12092), started 0:01:27 ago. (Use '!kill 12092' to kill it.)

**(Note):** tensorboard can be opened on "http://localhost:6006" when run locally

## Observation

We obtained maximum accuracy with the above model, with Test loss: 39.05% and Test accuracy: 86.07%.

In this case, 20% of the data was used for validation as indicated by `validation_split` in `model.fit`.

We also observed that adding `MaxPooling` layers, and increasing the number of `strides` reduces the time per epoch.

**Varying architecture with the below model**

```
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 strides=(2, 2),
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
```

Test Loss: 39.34%

Test accuracy: 85.67%

**Varying architecture with the below model**

```
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 strides=(2, 2),
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes, activation='softmax'))
```

Test loss: 38.58%

Test accuracy: 85.83%